Let say we want to say something about following problem:

$$
y' = Ay, y(0) \sim N(0, \sigma) , A \sim N(0,\sigma)
$$

The solution of this problem is a family of random variables $Y(t)$ the general way to obtain this numerically is sampling $y(0),A$ and solving a differential equation. In general there is no way around that $Y(t)$ just contains to much information in some cases you can use some tricks to make the problem possible and even analytically possible. <br>

But instead wanting to know the general solution we find it easier to make a Monte Carlo techniques that calculate things of the following form $E[f(Y(t))]$ where $f$ analytic. Great examples are $E[Y(t)]$ and $E[Y^{2}(t)]$ which contain some information about $Y(t)$. <br>

How is actually very simple:
$$
\begin{align*}
E[f(Y(t))] &= E[E[f(Y(t)) \mid y(0), A]] \\    
           &= E[E[f(y(t,y(0),A))]] \\    
            &= E[E[f(E[U(t,y(0),A)])]] \\
\end{align*}
$$
$y(t,y(0),A)$ is the solution of the ODE given $y(0)$ and $A$ and $U$ an unbiased estimator of it.
We already explained how to deal with $f$, see period1 analytic functions. <br> 

Ofcourse if $f$ isn't nice you can settle for approximations with analytical functions.